In [212]:
url = "https://www.govinfo.gov/app/collection/chrg/117/house/"
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import time
from rough_functions import * 
from new_functions import * 

In [219]:
data={}

text_links = []
for i in range(113,117+1):
    committees = []
    data[i*2+1787]={}
    url="https://www.govinfo.gov/app/collection/chrg/"+str(i)+"/house/"
    browser = webdriver.Chrome()
    browser.get(url)
    browser.implicitly_wait(10)
    over = (browser.find_elements(By.CLASS_NAME, 'panel-title'))
    for e in over:
        if(e.text[0] not in ["1","9","8","7"] and e.text[0:5]!='House' and e.text[0:5]!='Joint' and e.text[0:6]!='Senate'):
            committee = e.text
            data.get(i*2+1787)[e.text]={'url':url+committee.replace("'","%27").replace(' ',"%20").replace(",","%2C")}
            committees.append(committee)
    browser.close()
    for committee in committees:
        url=data.get(i*2+1787).get(committee).get('url')
        browser = webdriver.Chrome()
        browser.get(url)
        browser.implicitly_wait(10)
        text_buttons = (browser.find_elements(By.XPATH, "//a[@class='btn btn-sm btn-format']"))
        data.get(i*2+1787).get(committee)['text_link']=[]
        for t in text_buttons:
            if t.text=='TEXT':
                data.get(i*2+1787).get(committee).get('text_link').append(t.get_attribute("href"))
                text_links.append(t.get_attribute("href"))
        browser.close()


In [220]:
print(len(text_links))

7104


In [214]:
states = ["Northern Mariana Islands", "Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

In [237]:
list_of_dicts = []
for year in list(data.keys()):

    for k in list(data.get(year).keys()):
        if not 'appropriations' in k.lower() and not 'ethics' in k.lower():
            for link in data.get(year).get(k).get("text_link"):
                text  = requests.get(link).text
                if len(text.split('WASHINGTON : '))>1 and len(text.split('WASHINGTON : ')[1].split('OPENING STATEMENT')[0].split('C O N T E N T S')[0].split('CONTENTS')[0].split('Professional Staff')[0].split('COMMITTEE ON'))>1:
                    rough_committees = (text.split('WASHINGTON : ')[1].split('OPENING STATEMENT')[0].split('C O N T E N T S')[0].split('CONTENTS')[0].split('Professional Staff')[0].split('COMMITTEE ON')[1:])
                    for rough_com in rough_committees: 
                        committee_name = 'Committee On'+rough_com.split('\n',1)[0].lower().title()
                        # print(committee_name)
                        committee_members = rough_com.split('\n',1)[1]#.replace("`",'')
                        while '   ' in committee_members:
                            committee_members=committee_members.replace('   ','  ')
                        committee_members=committee_members.replace('  ','\n')
                        while '\n\n' in committee_members:
                            committee_members=committee_members.replace('\n\n','\n')
                        committee_members=committee_members.split('Subcommittee')[0]
                        
                    # print('<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>')
                        committee_members=committee_members.split('\n')
                        for c in range(len(committee_members)):
                            committee_member=committee_members[c]
                            committee_member=committee_member.split(', ')
                            if len(committee_member)<=1 and len(committee_member[0])>0 and (committee_member[0])[-1]==',' and c+2<len(committee_members):
                                committee_members[c]=(committee_members[c]+' '+((committee_members[c+2]))).split(', ')
                                committee_member=committee_members[c]
                            if len(committee_member)>1 and len(committee_member)<=3 and (any(substring in committee_member[1] for substring in states)):
                                single_data = {
                                    'full_name': committee_member[0].lower().title(),
                                    'state': committee_member[1].replace(',',''),
                                    'committee':committee_name,
                                    'year':year,
                                    'special':''
                                    }
                                if(committee_member[0].lower().split(' ')[-1] in ['x','xi','vii','vi','v','iv','iii','ii','i','jr.','sr.','jr','sr']):
                                    single_data['name']=committee_member[0].lower().split(' ')[-2]
                                else:
                                    single_data['name']=committee_member[0].lower().split(' ')[-1]
                                if len(committee_member)==3 and committee_member[2] not in ['',' ']:
                                    single_data['special']=committee_member[2]
                                list_of_dicts.append(single_data)




            # print(committee_members)

        # if (len(requests.get(data.get(2013).get(k).get("text_link")).text.lower().split(k.lower()+"\n\n"))) > 2:
        #     print(k)
        #     print(requests.get(data.get(2013).get(k).get("text_link")).text.lower().split(k.lower()+"\n\n")[2].split('\n\n')[0])


In [238]:
df = pd.json_normalize(list_of_dicts)
for state in states:
    df.loc[df['state'].str[0:len(state)] == state,'state']==state

df['count']=1
print(len(df))
df.head()



266500


,full_name,state,committee,year,special,name,count
0,Frank D. Lucas,Oklahoma,Committee On Agriculture,2013,Chairman,lucas,1
1,Bob Goodlatte,Virginia,Committee On Agriculture,2013,,goodlatte,1
2,Collin C. Peterson,Minnesota,Committee On Agriculture,2013,,peterson,1
3,Steve King,Iowa,Committee On Agriculture,2013,,king,1
4,Mike Mcintyre,North Carolina,Committee On Agriculture,2013,,mcintyre,1


In [308]:
df2 = df.groupby(['name','state','committee','year']).agg({'count': 'sum', 'full_name':pd.Series.mode})


df2=df2.reset_index()
df2['full_name']= df2['full_name'].astype(str)
for state in states:
    df2.loc[df2['state'].str[0:len(state)] == state,'state']=state
df2=df2.drop(df2[df2['state'].isin(states)==False].index,axis=0)
df2=df2.drop(df2[df2['name'].str.isnumeric()==True].index,axis=0)
df2 = df2.applymap(lambda x: x.strip() if isinstance(x, str) else x)


df2

,name,state,committee,year,count,full_name
0,,North Carolina,Committee On Small Business,2017,1,Alma
27,a.m.,Arizona,Committee On Oversight And Reform,2021,1,Received It At 5:40 A.M.
28,a.m.,Arizona,Committee On Oversight And Reform,2021,1,It Has Been Released At 5:40 A.M.
33,abraham,Louisiana,Committee On Agriculture,2015,63,Ralph Lee Abraham
34,abraham,Louisiana,Committee On Agriculture,2017,14,Ralph Lee Abraham
...,...,...,...,...,...,...
7135,zeldin,New York,Committee On Veterans' Affairs,2017,1,Lee Zeldin
7136,zeldin,New York,Committee On Foreign Affairs,2015,44,Lee M. Zeldin
7137,zinke,Montana,Committee On Armed Services,2015,38,Ryan K. Zinke
7138,zinke,Montana,Committee On Emerging Threats And Capabilities,2015,17,Ryan K. Zinke


In [309]:
df2['full_name']

0                                    Alma
27               Received It At 5:40 A.M.
28      It Has Been Released At 5:40 A.M.
33                      Ralph Lee Abraham
34                      Ralph Lee Abraham
                      ...                
7135                           Lee Zeldin
7136                        Lee M. Zeldin
7137                        Ryan K. Zinke
7138                        Ryan K. Zinke
7139                        Ryan K. Zinke
Name: full_name, Length: 6653, dtype: object

In [333]:
df3=df2.groupby(['name','state','year']).agg({'count':'sum','committee':lambda x: list(x), 'full_name':lambda x: max(list(x.mode()))})

df3=df3.reset_index()

df3.to_csv('house_committees_long.csv')
df3


,name,state,year,count,committee,full_name
0,,North Carolina,2017,1,[Committee On Small Business],Alma
1,a.m.,Arizona,2021,2,"[Committee On Oversight And Reform, Committee ...",Received It At 5:40 A.M.
2,abraham,Louisiana,2015,245,"[Committee On Agriculture, Committee On Disabi...",Ralph Abraham
3,abraham,Louisiana,2017,162,"[Committee On Agriculture, Committee On Armed ...",Ralph Lee Abraham
4,abraham,Louisiana,2019,95,"[Committee On Agriculture, Committee On Armed ...",Ralph Lee Abraham
...,...,...,...,...,...,...
2197,zeldin,New York,2015,398,[Committee On Disability Assistance And Memori...,Lee Zeldin
2198,zeldin,New York,2017,297,"[Committee On Financial Services, Committee On...",Lee M. Zeldin
2199,zeldin,New York,2019,218,"[Committee On Financial Services, Committee On...",Lee Zeldin
2200,zeldin,New York,2021,158,"[Committee On Financial Services, Committee On...",Lee Zeldin


In [322]:
(df3['full_name'].dtypes)

dtype('O')

In [283]:
df3['committee'].explode().value_counts()

Committee On Transportation And Infrastructure            399
Committee On Armed Services                               370
Committee On Financial Services                           363
Committee On Foreign Affairs                              350
Committee On Energy And Commerce                          303
                                                         ... 
Committee On Elections                                      4
Committee On Indian, Insular And Alaska Native Affairs      3
Committee On Economic And Consumer                          2
Committee On Science,                                       2
Committee On Intelligence                                   1
Name: committee, Length: 96, dtype: int64

In [284]:
df3

,name,state,year,count,committee
0,,North Carolina,2017,1,[Committee On Small Business]
1,a.m.,Arizona,2021,2,"[Committee On Oversight And Reform, Committee ..."
2,abraham,Louisiana,2015,245,"[Committee On Agriculture, Committee On Disabi..."
3,abraham,Louisiana,2017,162,"[Committee On Agriculture, Committee On Armed ..."
4,abraham,Louisiana,2019,95,"[Committee On Agriculture, Committee On Armed ..."
...,...,...,...,...,...
2197,zeldin,New York,2015,398,[Committee On Disability Assistance And Memori...
2198,zeldin,New York,2017,297,"[Committee On Financial Services, Committee On..."
2199,zeldin,New York,2019,218,"[Committee On Financial Services, Committee On..."
2200,zeldin,New York,2021,158,"[Committee On Financial Services, Committee On..."


In [217]:
out_data = pd.json_normalize(list_of_dicts)
out_data.to_csv('house_committees.csv')
out_data=pd.read_csv('house_committees.csv', usecols=['name','state','committee','year','special'])
out_data

,state,committee,year,special,name
0,Oklahoma,Committee On Agriculture,2013,Chairman,lucas
1,Virginia,Committee On Agriculture,2013,NaN,goodlatte
2,Minnesota,Committee On Agriculture,2013,NaN,peterson
3,Iowa,Committee On Agriculture,2013,NaN,king
4,North Carolina,Committee On Agriculture,2013,NaN,mcintyre
...,...,...,...,...,...
3606,Illinois,Committee On The Modernization Of Congress,2021,NaN,davis
3607,Georgia,Committee On The Modernization Of Congress,2021,NaN,williams
3608,Ohio,Committee On The Modernization Of Congress,2021,NaN,joyce
3609,Pennsylvania,Committee On The Modernization Of Congress,2021,NaN,reschenthaler
